In [1]:
# Работа с табличными данными
import pandas as pd
import numpy as np

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Преобразование признаков
from sklearn.preprocessing import MinMaxScaler

# Визуализация
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

## Знакомство с данными, предобработка

На некоторых этапах разбора предоставленных данных мы, вероятно, уже будем подвергать их некоторой преобработке преобразованиям, добавлением вспомогательных признаков и т.д. Поэтому уже на эпах знакомства с данными будет оформлять все манипуляции с данными в виде классов-преобразователей, чтобы в дальнейшем включить их в пайплайн.

### Кодировка жестов

- `Neutral/NOGO` (все нули) - нейтральное положение кисти, кисть в расслабленном состоянии

- `Thumb` (*Thumb* равен 1) - сгиб большого пальца

- `Grab` (*Thumb*, *Index*, *Middle*, *Ring*, *Pinky* равны 1) - схват, кисть сжата в кулак

- `Open` (*Thumb_stretch*, *Index_stretch*, *Middle_stretch*, *Ring_stretch*, *Pinky_stretch* равны 1) - открытая ладонь, пальцы выпрямлены

- `OK` (*Thumb*, *Index* равны 1) - жест “Окей”

- `Pistol` *(Middle*, *Ring*, *Pinky* равны 1) - жест “Пистолет”

In [ ]:
gest_labels = {
# Thumb Index Middle Ring Pinky Thumb_stretch Index_stretch Middle_stretch Ring_stretch Pinky_stretch
    '0000000000': 0, # 'NOGO'
    '1000000000': 1, # 'Thumb'
    '1111100000': 2, # 'Grab'
    '0000011111': 3, # 'Open'
    '1100000000': 4, # 'OK'
    '0011100000': 5  # 'Pistol'
}

### Дополнение метаданных

Условимся называть ***монтажом*** набор измерений, представленный в одном файле `.palm`.

Для чтения монтажей будем использовать предоставленную заказчиком функцию `read_omg_csv()`, установив дефолтные значения аргументов так, чтобы они сразу соответствовали актуальной структуре данных:

In [ ]:
def read_omg_csv(
    path_palm_data: str, 
    n_omg_channels: int = 50,    # '0', ..., '49' - каналы OMG датчиков 
    n_acc_channels: int = 3,     # 'ACC0', 'ACC1', 'ACC2' - акселерометр (потенциальные факторы)
    n_gyr_channels: int = 3,     # 'GYR0', 'GYR1', 'GYR2' - гироскоп (потенциальные факторы) 
    n_mag_channels: int = 0,     #  отсутствуют в данных
    n_enc_channels: int = 6,     # 'ENC0'...'ENC5' - не используются ???
    drop_enc: bool = True,       # нужно ли удалить столбцы 'ENC'
    button_ch: bool = True,      # 'BUTTON' - не используется
    drop_button: bool = True,    # нужно ли удалить столбец 'BUTTON'
    sync_ch: bool = True,        # 'SYNC' - синхронизация данных с протоколом
    timestamp_ch: bool = True,   # 'ts' - метка времени
    drop_timestamp: bool = False,# нужно ли удалить столбец 'ts'
    label_ch: bool = False       # присутствует ли столец с меткой (кодом) жеста
) -> pd.DataFrame:
    
    '''
    Reads CSV data for OMG data
    NB: data must be separated by " " separator

        Parameters:
                path_palm_data  (str): path to csv data file
                n_omg_channels  (int): Number of OMG channels
                n_acc_channels  (int): Number of Accelerometer channels, default = 0
                n_gyr_channels  (int): Number of Gyroscope channels, default = 0
                n_mag_channels  (int): Number of Magnetometer channels, default = 0
                n_enc_channels  (int): Number of Encoder channels, default = 0
                button_ch      (bool): If button channel is present, default = True
                sync_ch        (bool): If synchronization channel is present, default = True
                timestamp_ch   (bool): If timestamp channel is present, default = True
                label_ch       (bool): If label channel is present, default = False

        Returns:
                df_raw (pd.DataFrame): Parsed pandas Dataframe with OMG data
    '''
    
    df_raw = pd.read_csv(path_palm_data, sep=' ', 
                         header=None, 
                         skipfooter=1, 
                         skiprows=1, 
                         engine='python')
    columns = np.arange(n_omg_channels).astype('str').tolist()
    
    for label, label_count in zip(['ACC', 'GYR', 'MAG', 'ENC'], 
                                  [n_acc_channels, n_gyr_channels, n_mag_channels, n_enc_channels]):
        columns = columns + ['{}{}'.format(label, i) for i in range(label_count)]
        
    if button_ch:
        columns = columns + ['BUTTON']
        
    if sync_ch:
        columns = columns + ['SYNC']
        
    if timestamp_ch:
        columns = columns + ['ts']

    if label_ch:
        columns = columns + ['label']
        
    df_raw.columns = columns

    if drop_enc:
        enc_columns = [f"ENC{i}" for i in range(n_enc_channels)]
        df_raw.drop(enc_columns, axis=1, inplace=True)

    if drop_button:
        df_raw.drop('BUTTON', axis=1, inplace=True)

    if drop_timestamp:
        df_raw.drop('ts', axis=1, inplace=True)
    
    return df_raw

In [2]:
cols_omg = list(map(str, range(50)))

cols_gyr = [f'GYR{i}' for i in range(3)]

cols_acc = [f'ACC{i}' for i in range(3)]

Подгрузим предоставленное заказчиком описание (`meta_information.csv`) имеющихся данных:

In [ ]:
meta_info = pd.read_csv('data/meta_information.csv', index_col=1).drop('Unnamed: 0', axis=1)
display(meta_info.head(3))

print("Количество монтажей:", meta_info.shape[0])
print("Количество пилотов, с которых снимались данные:", meta_info['pilote_id'].nunique())

Дополним метаданные следующей информацией:

- периодичность измерений (*мс*) – разность между соседними метками времени

- количество измерений на один жест

- количество выполненных жестов

In [ ]:
# Поочередно для каждого набора данных добавим в метаданные:
for montage in meta_info.index:
    data = read_omg_csv(f"data/{montage}")
    protocol = pd.read_csv(f"data/{montage}.protocol.csv")
    
    # 1) периодичность измерений – разность между соседними метками времени
    ts_delta = (data['ts'].shift(-1) - data['ts']).value_counts().index.to_list()
    meta_info.loc[montage, 'ts_delta'] = ts_delta

    # 2) среднее кол-во измерений на один (не нейтральный) жест
    ticks_per_gest = data.groupby('SYNC')['ts'].count().median().round(2)
    meta_info.loc[montage, 'ticks_per_gest'] = ticks_per_gest

    # 3) кол-во выполненных жестов
    n_gestures = protocol.shape[0]
    meta_info.loc[montage, 'n_gestures'] = n_gestures

meta_info['pilote_id'] = meta_info['pilote_id'].astype(str)
meta_info.to_csv('montages/meta_info_extended.csv')

In [ ]:
print(f"Периодичность измерений: {'/'.join(map(str, meta_info['ts_delta'].unique()))} мс")
print(f"Медианное количество измерений на один жест: \
{', '.join(map(str, sorted(meta_info['ticks_per_gest'].astype(int).unique())))}")

display(meta_info.groupby('ticks_per_gest')['ticks_per_gest'].count().sort_index())

1. Периодичность измерений во всех монтажах одинакова и составляет 33 мс

2. В разных монтажах команды на выполнение жестов поступают с разной скоростью: от 19 до 61 измерения на один жест. При этом большинству монтажей соответствует скорость одна команда в секунду.

### Метки команд на выполнение жеста и пронации

Добавим в файлы с измерениями метки выполняемых жестов и метки пронации (по факту поступления команд в соответствии с протоколом).

In [ ]:
meta_info = pd.read_csv('montages/meta_info_extended.csv', index_col=0)

# Поочередно для каждого набора данных добавим в метаданные:
for montage in meta_info.index:

    data = read_omg_csv(f"data/{montage}")
    protocol = pd.read_csv(f"data/{montage}.protocol.csv")
    

    # метки жестов
    protocol['label'] = protocol.iloc[:, 1: 11].astype(int).astype(str).sum(axis=1).apply(lambda x: gest_labels[x])
    data = pd.merge(
        data, protocol[['epoch', 'label']], 
        how='left', left_on='SYNC', right_on='epoch'
    ).drop('epoch', axis=1)

    # метки пронации
    mask = protocol['Pronation'] != protocol['Pronation'].shift(1)
    protocol = protocol[mask]
    protocol['next_epoch'] = protocol['epoch'].shift(-1)
    protocol.iloc[-1, -1] = -1
    pron_epoches = np.array(protocol[['epoch', 'next_epoch', 'Pronation']]).tolist()

    for epoch_start, epoch_stop, pron in pron_epoches:
        start = data[data['SYNC'] == epoch_start].index[0]
        if epoch_stop == -1:
            stop = data.index[-1]
        else:
            stop = data[data['SYNC'] == epoch_stop].index[0]
        data.loc[start: stop, 'Pronation'] = {0: 0, 0.5: 1, 1: 2}[pron]


    data.to_csv(f"montages/{montage}", sep=',', index=None)

### Наличие показаний датчиков *GYR* и *ACC*

Также добавим в метаданные признаки наличия данных *GYR* и *ACC*

In [ ]:
meta_info = pd.read_csv('montages/meta_info_extended.csv', index_col=0)

meta_info['ACC'] = False
meta_info['GYR'] = False

for montage in meta_info.index:
    montage_info = meta_info.loc[montage]
    data = pd.read_csv('montages/' + montage_info.name)
    if any(data[cols_acc].std()):
        meta_info.loc[montage, 'ACC'] = True
    if any(data[cols_gyr].std()):
        meta_info.loc[montage, 'GYR'] = True

meta_info.to_csv('montages/meta_info_extended.csv')

### Датчики с высоким и низким уровнем сигнала

Посмотрим на некоторый интервал рядов показаний датчиков какого-либо монтажа:

In [ ]:
np.random.seed(42)
data = pd.read_csv('montages/' + np.random.choice(meta_info.index)).loc[:1000, cols_omg]
fig = px.line(data, width=1000, height=600)
fig.update_traces(line=dict(width=1))
fig.show()

Как можно убедиться, некоторые датчики выдают более *сильный* сигнал, чем другие. Причем сигналы датчиков *сильной* группы на первый взгляд кажутся более информативными, чем шумные и "невнятные" сигналы *слабых* датчиков.

Посмотрим на медианные значения датчиков в приведенном фрагменте данных:

In [ ]:
data_median = data.median()

fig = px.bar(data_median, width=1000, height=400)
fig.update_layout(showlegend=False)
fig.show()

print("Датчики с высоким уровнем сигнала:")
print(*data_median[data_median > data_median.mean()].index)


Однако в других монтажах ситуация может отличаться.

Чтобы проверить это, посмотрим на медианы показаний датчиков по всем монтажам:

In [ ]:
mont_medians = []
for montage in meta_info.index:
    data = pd.read_csv('montages/' + montage)[cols_omg]
    mont_medians.append(data.median().values.tolist())
mont_medians = np.array(mont_medians)
mont_medians = MinMaxScaler().fit_transform(mont_medians.T).T

fig_data = np.append(
    mont_medians,
    [mont_medians.min(axis=0), mont_medians.max(axis=0)], 
    axis=0
)

idx = [name.split('.')[0] + ' [plt ' + str(meta_info.loc[name, 'pilote_id']) + '] ' for name in meta_info.index]

fig_data = pd.DataFrame(
    fig_data,
    index=idx + ['минимум по монтажам ', 'максимум по монтажам '],
    columns=cols_omg
)

In [ ]:
fig = px.imshow(
    fig_data,
    width=1300, height=720,
    title="Медианы показаний датчиков по монтажам"
)
fig.update_coloraxes(showscale=False)
fig.update_layout(margin=dict(l=50, r=50, t=100, b=40), title_x=0.5)
fig.show()

Можно увидеть, что в разных монтажах разные датчики являются *сильными*.

При этом **нет ни одного датчика, который дает сильный сигнал во всех монтажах.**

Вместе с тем, некоторые датчики являются слабыми во всех монтажах:

In [ ]:
max_in_montages = fig_data.loc['максимум по монтажам ', :]
low_val_sensors = max_in_montages[max_in_montages < 0.2].index.tolist()
print(len(low_val_sensors), 'датчиков являются "слабыми" во всех монтажах:', *low_val_sensors)

Добавим в метаданные информацию о *сильных* датчиках для каждого монтажа. 
(к *сильным* будем относить датчики, медиана которых в нормализованных измерениях выше заданного порога `hi_val_threshold = 0.2`)

In [ ]:
hi_val_threshold = 0.2

lo_hi_sensors = pd.DataFrame(
    mont_medians,
    index=meta_info.index,
    columns=cols_omg
).T

meta_info['hi_val_sensors'] = None

for montage in lo_hi_sensors:
    mask = lo_hi_sensors[montage] > hi_val_threshold
    meta_info.at[montage, 'hi_val_sensors'] = lo_hi_sensors.loc[mask, montage].index.tolist()

meta_info.to_csv('montages/meta_info_extended.csv')

### Выбор монтажей для первого этапа работы

На первом этапе работы исключим монтажи, в которых:

- отсутствуют данные датчиков *GYR* и *ACC*;

- скорость подачи команды на смену жеста - чаще 1 раза в секунду.

In [ ]:
mask = meta_info['GYR'] & meta_info['ACC'] & (meta_info['ticks_per_gest'] >= 30)
meta_info = meta_info[mask]
meta_info.to_csv('marked/selected_montages.csv')

## Разметка обучающих данных по фактическому выполнению жестов

In [3]:
def find_train_gestures(
    data: pd.DataFrame,
    omg_cols: list,
    sync_col: str = 'SYNC',
    label_col: str = 'label',
    pron_col: str = 'Pronation',
    sync_shift = 0,
    window: int = 0,
    scale: bool = True,
    grad1_spacing: int = 5,
    grad2_spacing: int = 5,
    ingnore_n_left: int = 0,
    ignore_n_right: int = 0
) -> np.ndarray[int]:
    '''
    Осуществляет поиск границ фактически выполняемых жестов по локальным максимумам второго градиента измерений *omg*-датчиков. 

    ## Параметры
    **data**: *pd.DataFrame*<br>данные, включающие временные ряды показаний omg-датчиков

    **omg_cols**: *list*<br>список названий столбцов, соответствующих датчикам, по которым будут определены границы выполняемых жестов

    **sync_col**: *str, default="SYNC"*<br>Название признака, отвечающего за синхронизацию с протоколом

    **sync_shift**: *int, default=0*<br>Общий сдвиг разметки синхронизации

    **window**: *int, default=10*<br>ширина окна для предварительного сглаживания показаний датчиков по медиане; `0` - без сглаживания

    **scale**: *bool, default=True*<br>выполнять ли предварительное приведение показаний omg-датчиков к единому масштабу

    **grad1_spacing**: *int, default=5*<br>параметр `spacing` для функции `numpy.gradient()` для вычисления **первого** градиента

    **grad2_spacing**: *int, default=5*<br>параметр `spacing` для функции `numpy.gradient()` для вычисления **второго** градиента

    **ingnore_n_left**: *int, default=0*<br>не искать границу среди первых *ingnore_n_left* измерений метки синхронизации
    
    ## Возвращаемый результат

    Кортеж (**data_copy**, **bounds**)
    
    **data_copy**: *pandas.DataFrame*<br>Размеченная копия данных

    **bounds**: *numpy.ndarray[int]*<br>Номера строк (индексов), соответствующих границе выполняемых жестов.
    '''

    omg = np.array(data[omg_cols])

    # Сглаживание
    if window:
        omg = pd.DataFrame(omg).rolling(window, center=True).median()
    
    # Приведение к единому масштабу
    if scale:
        omg = MinMaxScaler((0, 1000)).fit_transform(omg)

    # Вычисление градиентов:
    # 1) Первый – сумма абсолютных градиентов по модулю
    grad1 = np.sum(np.abs(np.gradient(omg, grad1_spacing, axis=0)), axis=1)
    # не забудем заполнить образовавшиеся "дырки" из NaN
    grad1 = np.nan_to_num(grad1) 

    # 2) Второй – градиент первого градиента,
    grad2 = np.gradient(grad1, grad2_spacing)
    grad2 = np.nan_to_num(grad2)
    # усиленный возведением в квадрат, но с сохранением знака
    grad2 *= np.abs(grad2)

    # Искать локальные максимумы второго градиента будем внутри отрезков, 
    # определяемых по признаку синхронизации
    sync_mask = data[sync_col] != data[sync_col].shift(-1)
    sync_index = data[sync_mask].index + sync_shift

    res = []
    for l, r in zip(sync_index, sync_index[1:]):
        try:
            max_i = np.argmax(grad2[l + ingnore_n_left: r - ignore_n_right]) + ingnore_n_left
        except ValueError:
            break
        res.append((l + max_i, data.loc[l + max_i, label_col], data.loc[l + max_i, pron_col]))

    # Отдельно сохраним индексы границ жестов (мы их возвращаем в кортеже результата функции)
    bounds = np.array(res)[:, 0]

    # Теперь разметим каждое измерение в наборе фактическими метками
    data_copy = data.copy()
    data_copy['act_label'] = 0
    data_copy['act_pronation'] = 0
    for l, r in zip(res, res[1:] + [(data_copy.index[-1] + 1, 0, 0)]):
        data_copy.loc[l[0]: r[0], 'act_label'] = l[1]
        data_copy.loc[l[0]: r[0], 'act_pronation'] = l[2]


    return data_copy, bounds, grad2

Подгрузим метаданные о монтажах:

In [ ]:
meta_info = pd.read_csv('marked/selected_montages.csv', index_col=0)
# После чтения файла заново - столбец hi_val_sensors, 
# в котором хранятся списки активных датчиков, стал обычной строкой. 
# Нужно его преобразовать обратно в список
meta_info['hi_val_sensors'] = meta_info['hi_val_sensors'].apply(
    lambda x: x.strip('[]').replace("'", ' ').replace(' ', '').split(',')
)

Возьмем какой-нибудь монтаж:

In [ ]:
display(meta_info)
montage_info = meta_info.loc["2023-05-07_15-19-05.palm", :]

display(montage_info)

,pilote_id,last_train_idx,len(train),len(test),ts_delta,ticks_per_gest,n_gestures,ACC,GYR,hi_val_sensors
montage,,,,,,,,,,
2023-05-15_16-16-08.palm,1,23337,23337,5810,33.0,46.0,271.0,True,True,"[3, 4, 6, 12, 13, 16, 17, 21, 22, 27, 28, 30, ..."
2023-05-15_17-12-24.palm,1,23336,23336,5803,33.0,46.0,271.0,True,True,"[3, 4, 6, 12, 13, 16, 17, 21, 22, 27, 28, 30, ..."
2023-06-05_16-12-38.palm,1,17939,17939,4431,33.0,30.0,361.0,True,True,"[3, 4, 5, 6, 12, 13, 16, 17, 21, 22, 27, 28, 3..."
2023-06-05_17-53-01.palm,1,17771,17771,4435,33.0,31.0,361.0,True,True,"[3, 4, 5, 6, 12, 13, 16, 17, 21, 22, 27, 28, 3..."
2023-06-20_14-43-11.palm,1,17936,17936,4441,33.0,31.0,361.0,True,True,"[3, 4, 5, 6, 12, 13, 16, 17, 21, 27, 28, 30, 3..."
2023-06-20_13-30-15.palm,1,17928,17928,4435,33.0,31.0,361.0,True,True,"[3, 4, 5, 6, 12, 13, 16, 17, 21, 22, 27, 28, 3..."
2023-06-20_12-34-17.palm,1,17758,17758,4444,33.0,31.0,361.0,True,True,"[3, 4, 5, 6, 12, 13, 16, 17, 21, 22, 27, 28, 3..."
2023-09-30_08-06-44.palm,2,5693,5693,5509,33.0,31.0,181.0,True,True,"[7, 9, 10, 18, 20, 23, 26, 28, 31, 34, 37, 39]"
2023-09-29_11-03-50.palm,2,5694,5694,5511,33.0,31.0,181.0,True,True,"[7, 9, 10, 18, 20, 23, 26, 28, 31, 34, 37, 39]"


pilote_id                                                        3
last_train_idx                                                5361
len(train)                                                    5361
len(test)                                                     5884
ts_delta                                                      33.0
ticks_per_gest                                                61.0
n_gestures                                                    91.0
ACC                                                           True
GYR                                                           True
hi_val_sensors    [3, 4, 5, 6, 12, 13, 16, 17, 22, 27, 28, 38, 39]
Name: 2023-05-07_15-19-05.palm, dtype: object

In [15]:
data = pd.read_csv('montages/' + montage_info.name)

hi_val_sensors = montage_info['hi_val_sensors']

data_marked, bounds, grad2 = find_train_gestures(
    data, hi_val_sensors, 
    window=int(montage_info['ticks_per_gest'] // 4),
    grad1_spacing=int(montage_info['ticks_per_gest'] // 4),
    grad2_spacing=int(montage_info['ticks_per_gest'] // 4), 
    ingnore_n_left=3,
    ignore_n_right=int(montage_info['ticks_per_gest'] // 10)
)

In [7]:
def fig_marked_gestures(data_marked, bounds, montage_info, n=10_000, grad2=None):

    data = data_marked.iloc[:n, :].copy()

    data['label'] *= 100
    data['act_label'] *= 100

    hi_val_sensors = montage_info['hi_val_sensors']
    bounds = bounds.tolist()
    displayed_cols = hi_val_sensors + ['label', 'Pronation', 'act_label', 'act_pronation']

    if not grad2 is None:
        data['grad2'] = grad2[:n] * 200
        displayed_cols.append('grad2')

    fig = px.line(
        data[displayed_cols],
        width=1000, height=600,
        title=montage_info.name.split('.')[0],
        labels={'vaiable': ''}
    )
    fig.update_traces(line=dict(width=1))
    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_yaxes(visible=False)

    alt_fill = False
    for left, right in zip([0] + bounds, bounds):
        if all([left > n, right > n]):
            break
        fig.add_vrect(
            x0=left, x1=right, 
            line=dict(width=0), 
            fillcolor='gray', opacity=[0.1, 0.3][alt_fill],
            layer="below"
        )
        alt_fill = not alt_fill
        
    return fig

In [8]:
fig = fig_marked_gestures(data_marked, bounds, montage_info, 5000, grad2)
fig.show()

В целом результат нахождения фактических границ выполняемых жестов можно считать удовлетворительным. Из минусов можно отметить то, что момент начала смены жеста зачастую определяется с некоторым опозданием.

In [54]:
for montage in meta_info.index:

    montage_info = meta_info.loc[montage]
        
    hi_val_sensors = montage_info['hi_val_sensors']
    data = pd.read_csv("montages/" + montage)

    data_marked, bounds, _ = find_train_gestures(
        data, hi_val_sensors, 
        window=int(montage_info['ticks_per_gest'] // 4),
        grad1_spacing=int(montage_info['ticks_per_gest'] // 4),
        grad2_spacing=int(montage_info['ticks_per_gest'] // 4), 
        ingnore_n_left=3,
        ignore_n_right=int(montage_info['ticks_per_gest'] // 10)
    )

    data_marked.to_csv(f"marked/{montage}.marked", sep=',', index=None)
    pd.DataFrame(bounds).to_csv(f"marked/{montage}.bounds", sep=',', index=None)

    fig = fig_marked_gestures(data_marked, bounds, montage_info)
    fig.write_html("fig/" + montage + ".marked.html")

## Разделение данных на тренировочные и тестовые

In [19]:
meta_info = pd.read_csv('marked/selected_montages.csv', index_col=0)

for montage in meta_info.index:
    data = pd.read_csv('marked/' + montage + '.marked')
    last_train_idx = meta_info.loc[montage, 'last_train_idx']
    train = data.iloc[: last_train_idx + 1, :]
    test = data.iloc[last_train_idx + 1:, :]
    train.to_csv(f"marked/{montage}.train", sep=',')
    test.to_csv(f"marked/{montage}.test", sep=',')